##  Deprecated
must be updated to use the new package architecture

#### Symbolic Propagation fingerprint

SPF stands for Symbolic Propagation fingerprint, described [here](Symbolic_propagation_fingerprint.md)

first we need to calibrate the SPF model, by measuring the signal received in each room of the building. This is done by walking in each room, and measuring the signal received by the CartoTag. The CartoTag is held above the head, to limit the absorption of the signal by the body of the wearer. The wearer of the CartoTag moves within the room, which allows to have a representation of the absorption for the whole room and not only for the position at which the wearer stayed.

the calibration part will be done in an app and using a proper database in the future if the model is efficient enough.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from src.API import API
import plotly.graph_objects as go

from src.DataLoader import gdfLoader,ReceiverLoader,RSSIDataLoader


In [3]:
#Calibration time setting

start,end="2023-06-22 10:40:00.000000","2023-06-22 11:30:35.000000"
time={
    "21855":("2023-06-22 10:41:00.000000","2023-06-22 10:44:00.000000"),#4A435
    "21279":("2023-06-22 10:46:00.000000","2023-06-22 10:49:00.000000"),#couloir
    "21267":("2023-06-22 10:51:00.000000","2023-06-22 10:54:00.000000"),#ascenseur
    "21821":("2023-06-22 10:56:00.000000","2023-06-22 11:03:00.000000"),#4A436
    "21820":("2023-06-22 11:06:00.000000","2023-06-22 11:09:00.000000"),#4A438
}

In [4]:
ModuleMapPath="data/carteModulePalaiseau.csv"
marker_gdf=ReceiverLoader(ModuleMapPath)
marker_gdf

,macModule,geometry,coordinatesEPSG3857
0,A8032A31204E,POINT (2.20079 48.71316),POINT (244990.444 6226329.680)
1,C45BBE39F42A,POINT (2.20110 48.71315),POINT (245025.333 6226329.187)
2,C45BBE39F9AE,POINT (2.20120 48.71314),POINT (245035.947 6226326.828)
3,A8032A311FAA,POINT (2.20072 48.71317),POINT (244983.036 6226331.224)
4,C45BBE39F56A,POINT (2.20117 48.71318),POINT (245033.645 6226333.296)
5,C45BBE37B346,POINT (2.20087 48.71315),POINT (244999.412 6226328.251)
6,C45BBE386952,POINT (2.20098 48.71316),POINT (245011.849 6226330.207)
7,C45BBE386362,POINT (2.20094 48.71314),POINT (245007.682 6226326.760)
8,C45BBE384CD2,POINT (2.20093 48.71317),POINT (245006.234 6226331.144)
9,C45BBE39870E,POINT (2.20106 48.71315),POINT (245020.561 6226328.967)


In [5]:
# data gathering
callAPI=API()
hour_correction=2
start=pd.to_datetime(start)-pd.Timedelta(hour_correction, unit="h")
end=pd.to_datetime(end)-pd.Timedelta(hour_correction, unit="h")
data=callAPI.getRawDataForCartoWear('C77C2F92664E',start,end)
data=pd.DataFrame(data)
data["timestamp"]=pd.to_datetime(data['timestamp'])



In [6]:
room_data={}
for room in time:
    room_data[room]=data[(data['timestamp'] > pd.to_datetime(time[room][0])) & (data['timestamp'] < pd.to_datetime(time[room][1]))]
    room_data[room]=room_data[room].reset_index(drop=True)
print(room_data)

{'21855': Empty DataFrame
Columns: [timestamp, macModule, rssi]
Index: [], '21279': Empty DataFrame
Columns: [timestamp, macModule, rssi]
Index: [], '21267': Empty DataFrame
Columns: [timestamp, macModule, rssi]
Index: [], '21821': Empty DataFrame
Columns: [timestamp, macModule, rssi]
Index: [], '21820': Empty DataFrame
Columns: [timestamp, macModule, rssi]
Index: []}


In [7]:
outlier_suppress=0.35
#define macmodule expected rssi value for each room
macmodule_expected_rssi={}
for room in room_data:
    macmodule_expected_rssi[room]={}
    # for the room x i received signal from the macmodule list
    macmodule_list=room_data[room].macModule.unique()
    for macmodule in macmodule_list:
        # for each macmodule i received signal from the room x list
        rssi_list_for_macmodule=room_data[room][room_data[room].macModule==macmodule].rssi
        macmodule_expected_rssi[room][macmodule]=(rssi_list_for_macmodule.quantile(outlier_suppress),rssi_list_for_macmodule.quantile(1-outlier_suppress))
macmodule_expected_rssi    


{'21855': {}, '21279': {}, '21267': {}, '21821': {}, '21820': {}}

In [8]:
macmodule_list=marker_gdf.macModule.tolist()
macmodule_expected_room={}
for macModule in macmodule_list:
    macmodule_expected_room[macModule]={}
    for room in macmodule_expected_rssi:
        macmodule_expected_room[macModule][room]={}
        if macModule in macmodule_expected_rssi[room]:
            macmodule_expected_room[macModule][room]=macmodule_expected_rssi[room][macModule]
macmodule_expected_room


{'A8032A31204E': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE39F42A': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE39F9AE': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'A8032A311FAA': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE39F56A': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE37B346': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE386952': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE386362': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE384CD2': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'C45BBE39870E': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '21820': {}},
 'A8032A312052': {'21855': {},
  '21279': {},
  '21267': {},
  '21821': {},
  '2

In [9]:
import json
with open("data/spf_calibration/calib1.json", "w") as file:
    json.dump(macmodule_expected_room, file)
    file.close()